# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import random
from sqlalchemy import create_engine
import psycopg2


In [2]:
# Create a connection to the database
from sqlalchemy.engine import URL
url_object = URL.create(
    "postgresql",
    username="postgres",
    password="Saad@2017",  # plain (unescaped) text
    host="localhost",
    database="fraud_detection",
)

# Create the engine object
engine = create_engine(url_object)

query1 = """
SELECT card_holder.id AS "cardholder_id", card_holder.name AS "name", t.id AS "transaction_id", 
t.amount AS "amount",t.date :: timestamp :: date AS "date", 
t.date :: timestamp :: time AS "time"
FROM transaction as t
INNER JOIN credit_card as cc ON t.card=cc.card
INNER JOIN card_holder ON cc.cardholder_id = card_holder.id

        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
customer_transaction_df=pd.read_sql(query1, engine)
customer_transaction_df

,cardholder_id,name,transaction_id,amount,date,time
0,13,John Martin,222,6.22,2018-01-01,21:35:10
1,13,John Martin,2045,3.83,2018-01-01,21:43:12
2,10,Matthew Gutierrez,395,9.61,2018-01-01,22:41:21
3,4,Danielle Green,3309,19.03,2018-01-01,23:13:30
4,18,Malik Carlson,567,2.95,2018-01-01,23:15:10
...,...,...,...,...,...,...
3495,15,Kyle Tucker,1979,4.84,2018-12-31,01:24:15
3496,10,Matthew Gutierrez,2342,3.26,2018-12-31,03:33:28
3497,19,Peter Mckay,948,10.73,2018-12-31,05:53:58
3498,23,Mark Lewis,1168,11.87,2018-12-31,08:22:17


In [3]:
# Write function that locates outliers using standard deviation
def outliers_std(card_id):
    
    # get transaction amounts for card id 
    transaction_amounts_df = customer_transaction_df.loc[customer_transaction_df['cardholder_id']==card_id, ['cardholder_id','amount']]
    # calculate summary statistics
    data_mean= transaction_amounts_df["amount"].mean()
    data_std= transaction_amounts_df["amount"].std()
    # identify outliers
    cut_off = data_std * 3
    lower= data_mean - cut_off
    upper= data_mean + cut_off   
    outlier_df=transaction_amounts_df[(transaction_amounts_df["amount"] < lower) | (transaction_amounts_df["amount"] >  upper)] 
 
    return outlier_df

# how to calculate outliers
# https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

In [7]:
# Find anomalous transactions for 3 random card holders

rand_card_id = np.random.randint(1,25,3)

for item in rand_card_id:
    if outliers_std(item).empty:  # check if returned data frame is empty
        print(f"Card holder {item} has no outlier transactions.")
    else:
        print(f"Cardholder_id {item} has the following outliers.:\n {outliers_std(item)}")

Card holder 8 has no outlier transactions.
Card holder 10 has no outlier transactions.
Cardholder_id 1 has the following outliers.:
       cardholder_id  amount
235               1  1691.0
2079              1  1302.0
2374              1  1790.0
3191              1  1660.0
3263              1  1894.0


## Identifying Outliers Using Interquartile Range

In [5]:
# Write a function that locates outliers using interquartile range
def outliers_interq(card_id):
    
    # get transaction amounts for card id 
    transaction_amounts_df = customer_transaction_df.loc[customer_transaction_df['cardholder_id']==card_id, ['cardholder_id','amount']]
    # calculate summary statistics
    q25 = transaction_amounts_df["amount"].quantile(0.25)
    q75= transaction_amounts_df["amount"].quantile(0.75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower = q25 - cut_off
    upper=q75 + cut_off

    outlier_interq_df=transaction_amounts_df[(transaction_amounts_df["amount"] < lower) | (transaction_amounts_df["amount"] >  upper)] 
 
    return  outlier_interq_df

# how to calculate outliers
# https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

In [6]:
# Find anomalous transactions for 3 random card holders
# Find anomalous transactions for 3 random card holders

rand_card_id = np.random.randint(1,25,3)

for item in rand_card_id:
    if  outliers_interq(item).empty:  # check if returned data frame is empty
        print(f"Card holder {item} has no interquantile  outlier transactions.")
    else:
        print(f"Cardholder_id {item} has the following interquantile outliers.:\n {outliers_std(item)}")

Cardholder_id 16 has the following interquantile outliers.:
       cardholder_id  amount
212              16  1131.0
457              16  1430.0
625              16  1617.0
1454             16  1203.0
1638             16  1103.0
2043             16  1803.0
3067             16  1911.0
3237             16  1014.0
3429             16  1634.0
Card holder 8 has no interquantile  outlier transactions.
Cardholder_id 7 has the following interquantile outliers.:
       cardholder_id  amount
27                7  1685.0
484               7  1072.0
1079              7  1086.0
2142              7  1449.0
3327              7  2249.0
3379              7  1296.0
